In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName("SQL_Spark").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/26 20:13:40 WARN Utils: Your hostname, bhuvaneshwaran-Latitude-5420, resolves to a loopback address: 127.0.1.1; using 192.168.1.17 instead (on interface wlp0s20f3)
25/12/26 20:13:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/26 20:13:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
df=spark.read.parquet(r"/home/bhuvaneshwaran/Desktop/Medium/parquetFiles/Employees.parquet")
df.show(5)

+---+----------------+-------------------+--------------------+--------------------+-----------------+-----+-----------+------+--------------------+-------+--------------------+------------------+--------------------+----------+-------------+
| id|            name|          full_name|             address|         json_string|dob_string_format|score| department|salary|        hobbies_list|user_id|            txn_date|         txn_value|              street|      city|        state|
+---+----------------+-------------------+--------------------+--------------------+-----------------+-----+-----------+------+--------------------+-------+--------------------+------------------+--------------------+----------+-------------+
|  1|    Advik Halder|         Janya Bala|H.No. 916, Choksh...|{"city": "Bengalu...|       1974-10-29|   74|      Sales| 95186|       [Photography]|      2|2025-11-29 13:20:...|132.71083639903748|   20/662\nDua Nagar|Coimbatore|    Rajasthan|
|  2|     Omaja Baria|      

## 🔑 Before Writing SQL: One Mandatory Step

    * Spark SQL works on tables or views, not directly on DataFrames.

    * So first, we must convert a DataFrame into a temporary view.

## ✅ Step 1: Create a Temporary View

## createOrReplaceTempView

This method:

    * Registers the DataFrame as a temporary SQL table
    
    * Makes it accessible using SQL syntax

In [5]:
df.createOrReplaceTempView("employees")

In [13]:
SELECT * FROM employees

## ❓ Why “Temporary” View?

Because:

    * It exists only for the current SparkSession
    
    * Once Spark stops → the view disappears
    
    * It is stored in memory, not on disk

## 🔄 Alternative Method

In [11]:
df.createTempView("employees")

Difference:

    * createTempView ❌ fails if view already exists
    
    * createOrReplaceTempView ✅ safely replaces it
    
    * 👉 Best practice: Always use createOrReplaceTempView

## 🧪 Step 2: Write Your First Spark SQL Query

## .sql()

This method:

    * Accepts a SQL query as a string
    
    * Sends it to Spark’s SQL engine
    
    * Converts it internally into an optimized execution plan

In [15]:
spark.sql("SELECT * FROM employees").show(2)

+---+------------+------------+--------------------+--------------------+-----------------+-----+----------+------+--------------------+-------+--------------------+------------------+--------------------+----------+-------------+
| id|        name|   full_name|             address|         json_string|dob_string_format|score|department|salary|        hobbies_list|user_id|            txn_date|         txn_value|              street|      city|        state|
+---+------------+------------+--------------------+--------------------+-----------------+-----+----------+------+--------------------+-------+--------------------+------------------+--------------------+----------+-------------+
|  1|Advik Halder|  Janya Bala|H.No. 916, Choksh...|{"city": "Bengalu...|       1974-10-29|   74|     Sales| 95186|       [Photography]|      2|2025-11-29 13:20:...|132.71083639903748|   20/662\nDua Nagar|Coimbatore|    Rajasthan|
|  2| Omaja Baria|Manya Bhakta|492, Dass Road, B...|{"city": "Coimbat...|   

## "SELECT * FROM employees"

Standard SQL syntax:

    * SELECT * → select all columns
    
    * FROM employees → the temp view name

## 🔥 Filtering Data Using Spark SQL

In [16]:
df.columns

['id',
 'name',
 'full_name',
 'address',
 'json_string',
 'dob_string_format',
 'score',
 'department',
 'salary',
 'hobbies_list',
 'user_id',
 'txn_date',
 'txn_value',
 'street',
 'city',
 'state']

In [17]:
spark.sql("""
    SELECT name, salary
    FROM employees
    WHERE salary > 50000""").show()


+--------------------+------+
|                name|salary|
+--------------------+------+
|        Advik Halder| 95186|
|         Omaja Baria| 54210|
|        Unni Iyengar|113058|
|    Manthan Kulkarni|149570|
|          Ayaan Rege| 78937|
|      Aarush Sanghvi|145117|
|Ikshita Radhakris...|136837|
|      Upadhriti Tata|123045|
|          Omya Verma| 76844|
|         Tejas Kanda|121007|
|        Jairaj Walla|114744|
|        Jeet Chaudry| 76061|
|       Arunima Uppal|100911|
|    Upadhriti Sachar| 99075|
|     Bhanumati Sethi|119373|
|         Darsh Chada|126499|
|          Rishi Bail|105467|
|       Madhavi Ghose|117193|
|      Owen Ramaswamy| 74949|
|        Watika Banik| 64731|
+--------------------+------+
only showing top 20 rows


## Same as 

In [21]:
df.filter(df.salary > 50000).select("name","salary").show()

+--------------------+------+
|                name|salary|
+--------------------+------+
|        Advik Halder| 95186|
|         Omaja Baria| 54210|
|        Unni Iyengar|113058|
|    Manthan Kulkarni|149570|
|          Ayaan Rege| 78937|
|      Aarush Sanghvi|145117|
|Ikshita Radhakris...|136837|
|      Upadhriti Tata|123045|
|          Omya Verma| 76844|
|         Tejas Kanda|121007|
|        Jairaj Walla|114744|
|        Jeet Chaudry| 76061|
|       Arunima Uppal|100911|
|    Upadhriti Sachar| 99075|
|     Bhanumati Sethi|119373|
|         Darsh Chada|126499|
|          Rishi Bail|105467|
|       Madhavi Ghose|117193|
|      Owen Ramaswamy| 74949|
|        Watika Banik| 64731|
+--------------------+------+
only showing top 20 rows


| SQL                   | DataFrame           |
| --------------------- | ------------------- |
| Easier to read        | More Pythonic       |
| Familiar to SQL users | Better for chaining |
| Great for analysts    | Great for engineers |


## 🔥 Aggregations Using Spark SQL

In [22]:
spark.sql("""
    SELECT department, AVG(salary) AS avg_salary
    FROM employees
    GROUP BY department
""").show()


+-----------+-----------------+
| department|       avg_salary|
+-----------+-----------------+
|      Sales|89892.16936488169|
|Engineering|89688.36086529007|
|         HR|91034.83843452082|
|    Finance|89672.33032355155|
|  Marketing|89144.24492900609|
+-----------+-----------------+



## Equivalent DataFrame Code

In [27]:
from pyspark.sql.functions import avg
df.select("department",'salary').show(2)
df.groupBy("department").agg(avg("salary").alias("avg_salary")).show()

+----------+------+
|department|salary|
+----------+------+
|     Sales| 95186|
|   Finance| 54210|
+----------+------+
only showing top 2 rows
+-----------+-----------------+
| department|       avg_salary|
+-----------+-----------------+
|      Sales|89892.16936488169|
|Engineering|89688.36086529007|
|         HR|91034.83843452082|
|    Finance|89672.33032355155|
|  Marketing|89144.24492900609|
+-----------+-----------------+



## 🔥 Sorting Results

In [28]:
spark.sql("""
    SELECT name, salary
    FROM employees
    ORDER BY salary DESC
""").show()


+-------------------+------+
|               name|salary|
+-------------------+------+
|       Ronith Divan|149992|
|  Jagvi Chakraborty|149987|
| Kevin Mukhopadhyay|149971|
|        Kamya Amble|149967|
|       Anjali Boase|149960|
|      Turvi Kadakia|149960|
|        Yutika Dash|149960|
|       Charvi Amble|149958|
|  Charvi Srinivasan|149940|
|       Yahvi Sharaf|149937|
|      Wriddhish Lad|149932|
|        Kalpit Yogi|149912|
|Madhavi Rajagopalan|149905|
|     Tanmayi Sekhon|149902|
|        Zayyan Dara|149897|
| Chandani Nagarajan|149897|
|      Aarini Kurian|149882|
|           Sara Din|149878|
|     Dakshesh Chana|149868|
|       Aashi Bhagat|149862|
+-------------------+------+
only showing top 20 rows


In [29]:
spark.sql("""
    SELECT department, AVG(salary) AS avg_salary
    FROM employees
    GROUP BY department
    order by avg_salary desc
""").show()


+-----------+-----------------+
| department|       avg_salary|
+-----------+-----------------+
|         HR|91034.83843452082|
|      Sales|89892.16936488169|
|Engineering|89688.36086529007|
|    Finance|89672.33032355155|
|  Marketing|89144.24492900609|
+-----------+-----------------+



DESC

Descending order.

Alternative:

ASC → ascending (default)

In [30]:
spark.sql("""
    SELECT department, AVG(salary) AS avg_salary
    FROM employees
    GROUP BY department
    order by avg_salary asc
""").show()


+-----------+-----------------+
| department|       avg_salary|
+-----------+-----------------+
|  Marketing|89144.24492900609|
|    Finance|89672.33032355155|
|Engineering|89688.36086529007|
|      Sales|89892.16936488169|
|         HR|91034.83843452082|
+-----------+-----------------+



## 🔥 LIMIT Clause

In [33]:
spark.sql("""
    SELECT *
    FROM employees
    LIMIT 2
""").show()

+---+------------+------------+--------------------+--------------------+-----------------+-----+----------+------+--------------------+-------+--------------------+------------------+--------------------+----------+-------------+
| id|        name|   full_name|             address|         json_string|dob_string_format|score|department|salary|        hobbies_list|user_id|            txn_date|         txn_value|              street|      city|        state|
+---+------------+------------+--------------------+--------------------+-----------------+-----+----------+------+--------------------+-------+--------------------+------------------+--------------------+----------+-------------+
|  1|Advik Halder|  Janya Bala|H.No. 916, Choksh...|{"city": "Bengalu...|       1974-10-29|   74|     Sales| 95186|       [Photography]|      2|2025-11-29 13:20:...|132.71083639903748|   20/662\nDua Nagar|Coimbatore|    Rajasthan|
|  2| Omaja Baria|Manya Bhakta|492, Dass Road, B...|{"city": "Coimbat...|   

### Why use LIMIT?

    * Preview data
    
    * Debug queries
    
    * Avoid large outputs

## 🔥 Joins Using Spark SQL

In [34]:
df.columns

['id',
 'name',
 'full_name',
 'address',
 'json_string',
 'dob_string_format',
 'score',
 'department',
 'salary',
 'hobbies_list',
 'user_id',
 'txn_date',
 'txn_value',
 'street',
 'city',
 'state']

In [44]:
depts=df.select("id","department","salary")

In [45]:
depts.columns

['id', 'department', 'salary']

In [39]:
depts.createOrReplaceTempView("departments")

In [40]:
spark.sql("""
    SELECT e.name, d.department
    FROM employees e
    INNER JOIN departments d
    ON e.id = d.id
""").show()


+--------------------+-----------+
|                name| department|
+--------------------+-----------+
|        Advik Halder|      Sales|
|         Omaja Baria|    Finance|
|        Unni Iyengar|         HR|
|    Manthan Kulkarni|Engineering|
|          Ayaan Rege|Engineering|
|      Aarush Sanghvi|      Sales|
|      Tristan Sahota|      Sales|
|Ikshita Radhakris...|    Finance|
|      Upadhriti Tata|    Finance|
|         Arya Sharma|Engineering|
|          Omya Verma|Engineering|
|         Tejas Kanda|  Marketing|
|        Jairaj Walla|    Finance|
|        Jeet Chaudry|  Marketing|
|       Arunima Uppal|  Marketing|
|    Upadhriti Sachar|         HR|
|     Bhanumati Sethi|  Marketing|
|         Darsh Chada|      Sales|
|          Rishi Bail|    Finance|
|       Madhavi Ghose|         HR|
+--------------------+-----------+
only showing top 20 rows


**employees e**

    Alias for employees table.

**departments d**

    Alias for departments table.

**INNER** JOIN

    Join type.

Other options:

    * LEFT JOIN
    
    * RIGHT JOIN
    
    * FULL JOIN

ON e.id = d.id

    Join condition.

## 🔥 Using CASE WHEN in Spark SQL

In [49]:
spark.sql("""
    SELECT name,department,salary,
           CASE
               WHEN salary > 80000 THEN 'High'
               WHEN salary > 50000 THEN 'Medium'
               ELSE 'Low'
           END AS salary_category
    FROM employees
""").show()


+--------------------+-----------+------+---------------+
|                name| department|salary|salary_category|
+--------------------+-----------+------+---------------+
|        Advik Halder|      Sales| 95186|           High|
|         Omaja Baria|    Finance| 54210|         Medium|
|        Unni Iyengar|         HR|113058|           High|
|    Manthan Kulkarni|Engineering|149570|           High|
|          Ayaan Rege|Engineering| 78937|         Medium|
|      Aarush Sanghvi|      Sales|145117|           High|
|      Tristan Sahota|      Sales| 34016|            Low|
|Ikshita Radhakris...|    Finance|136837|           High|
|      Upadhriti Tata|    Finance|123045|           High|
|         Arya Sharma|Engineering| 36329|            Low|
|          Omya Verma|Engineering| 76844|         Medium|
|         Tejas Kanda|  Marketing|121007|           High|
|        Jairaj Walla|    Finance|114744|           High|
|        Jeet Chaudry|  Marketing| 76061|         Medium|
|       Arunim

Equivalent DataFrame Logic

In [62]:
from pyspark.sql.functions import when

df.withColumn(
    "Sal_Cat",
    when(df.salary > 80000, "High").
    when(df.salary > 50000, "Medium").otherwise("Low")
).select("department","salary","Sal_Cat").show(5)

+-----------+------+-------+
| department|salary|Sal_Cat|
+-----------+------+-------+
|      Sales| 95186|   High|
|    Finance| 54210| Medium|
|         HR|113058|   High|
|Engineering|149570|   High|
|Engineering| 78937| Medium|
+-----------+------+-------+
only showing top 5 rows


## 🔥 Checking Execution Plan (Very Important)

In [63]:
spark.sql("SELECT * FROM employees WHERE salary > 50000").explain()

== Physical Plan ==
*(1) Filter (isnotnull(salary#8L) AND (salary#8L > 50000))
+- *(1) ColumnarToRow
   +- FileScan parquet [id#0L,name#1,full_name#2,address#3,json_string#4,dob_string_format#5,score#6L,department#7,salary#8L,hobbies_list#9,user_id#10L,txn_date#11,txn_value#12,street#13,city#14,state#15] Batched: true, DataFilters: [isnotnull(salary#8L), (salary#8L > 50000)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/home/bhuvaneshwaran/Desktop/Medium/parquetFiles/Employees.parquet], PartitionFilters: [], PushedFilters: [IsNotNull(salary), GreaterThan(salary,50000)], ReadSchema: struct<id:bigint,name:string,full_name:string,address:string,json_string:string,dob_string_format...




## What .explain() Does

    Shows how Spark will execute the query

## Reveals:

    * Filters
    
    * Shuffles
    
    * Joins
    
    * Optimizations

## Why beginners should learn this early?

Because:

    * You’ll understand slow queries
    
    * You’ll avoid expensive operations
    
    * You’ll think like a Spark engineer

## ⚖️ When to Use Spark SQL vs DataFrame API
Use Spark SQL when:

    * Logic is complex
    
    * Team knows SQL well
    
    * Business rules change often
    
    * Queries are long and readable

Use DataFrame API when:

    * Heavy transformations
    
    * Python-based logic
    
    * Dynamic pipelines
    
    * Reusable functions

👉 Best engineers know both.

## 🎯 Summary of This Blog

You now understand:

    * What Spark SQL is
    
    * How temporary views work
    
    * How spark.sql() works internally
    
    * Every SQL clause used
    
    * SQL vs DataFrame tradeoffs
    
    * How Spark optimizes SQL queries
    
    * Why .explain() is critical

This blog fills a huge knowledge gap for beginners.